In [9]:
import torch
import torch.nn as nn

### LeNet Implementation for gray-scale image (in_channel = 1) with size (28,28)

In [10]:
class LeNet(nn.Module):
    def __init__(self, in_channels, lr=0.1, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            # Convolution layer 1
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=6,
                kernel_size=5,
                padding=2,
            ),
            nn.Sigmoid(),
            # Pooling layer 1
            nn.AvgPool2d(kernel_size=2, stride=2),
            # Convolution layer 2
            nn.Conv2d(
                in_channels=6,
                out_channels=16,
                kernel_size=5
            ),
            nn.Sigmoid(),
            # Pooling layer 2
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(), # For switch between convolution to linear layer 
            # FC1 (Fully Connected/Linear Layer 1)
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.Sigmoid(),
            # FC2 (Fully Connected/Linear Layer 2)
            nn.Linear(in_features=120, out_features=84,),
            nn.Sigmoid(),
            # FC3 (Final Linear layer with outfeatures = num of classes)
            nn.Linear(in_features=84, out_features=num_classes)


        )

    def forward(self, x):
        return self.net(x)
    
    def layer_summary(self, x):
        for layer in self.net:
            x = layer(x)
            if isinstance(layer, (nn.Conv2d, nn.Linear, nn.AvgPool2d, nn.Flatten)):
                print(f"{layer.__class__.__name__}: {x.shape}")

In [11]:
in_channels = 1
num_classes = 10
model = LeNet(in_channels=in_channels, num_classes=num_classes)
model

LeNet(
  (net): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [12]:
X = torch.randn((1, 1, 28, 28))
print(X.shape,'\n\n')
model.layer_summary(X)

torch.Size([1, 1, 28, 28]) 


Conv2d: torch.Size([1, 6, 28, 28])
AvgPool2d: torch.Size([1, 6, 14, 14])
Conv2d: torch.Size([1, 16, 10, 10])
AvgPool2d: torch.Size([1, 16, 5, 5])
Flatten: torch.Size([1, 400])
Linear: torch.Size([1, 120])
Linear: torch.Size([1, 84])
Linear: torch.Size([1, 10])


### LeNet Implementation Independent of Image Size

In [13]:
class DynamicLeNet(nn.Module):
    def __init__(self, in_channels, image_size, lr=0.1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=6,
            kernel_size=5,
            padding=2
        )
        self.sigmoid1 = nn.Sigmoid()
        self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=5

        )
        self.sigmoid2 = nn.Sigmoid()
        self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.convlayer = nn.Sequential(
            self.conv1,
            self.sigmoid1,
            self.avgpool1,
            self.conv2,
            self.sigmoid2,
            self.avgpool2
        )

        self.flattened_size = self.convlayer(torch.zeros(
            1, in_channels, image_size, image_size)).shape[1]

        self.fc1 = nn.Linear(in_features=self.flattened_size, out_features=120)
        self.sigmoid3 = nn.Sigmoid()
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.sigmoid4 = nn.Sigmoid()
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)

        self.fullyconnected = nn.Sequential(
            self.fc1, self.sigmoid3, self.fc2, self.sigmoid4, self.fc3
        )

    def forward(self, x):
        x = self.convlayer(x)
        x = x.view(-1, self.flattened_size)
        x = self.fullyconnected(x)
        return x
    
    def layer_summary(self, x):
        for layer in self.convlayer:
            x = layer(x)
            if isinstance(layer, (nn.Conv2d, nn.Linear, nn.AvgPool2d)):
                print(f"{layer.__class__.__name__}: {x.shape}")
        x = x.view(-1, self.flattened_size)
        print(f"Flatten Layer: {self.flattened_size}")
        for layer in self.fullyconnected:
            x = layer(x)
            if isinstance(layer, nn.Linear):
                print(f"{layer.__class__.__name__}: {x.shape}")



In [14]:
in_channels = 1
num_classes = 10
image_size = 32
model = DynamicLeNet(in_channels=in_channels,
                     num_classes=num_classes,
                     image_size=image_size,)
model

DynamicLeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (sigmoid1): Sigmoid()
  (avgpool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (sigmoid2): Sigmoid()
  (avgpool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convlayer): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=16, out_features=120, bias=True)
  (sigmoid3): Sigmoid()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (sigmoid4): Sigmoid()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fullyconnected): Sequential(
    (0): Linear(in_features=16, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120

In [15]:
X = torch.randn((1, 1, image_size, image_size))
print(X.shape, '\n')
model.layer_summary(X)

torch.Size([1, 1, 32, 32]) 

Conv2d: torch.Size([1, 6, 32, 32])
AvgPool2d: torch.Size([1, 6, 16, 16])
Conv2d: torch.Size([1, 16, 12, 12])
AvgPool2d: torch.Size([1, 16, 6, 6])
Flatten Layer: 16
Linear: torch.Size([36, 120])
Linear: torch.Size([36, 84])
Linear: torch.Size([36, 10])
